In [27]:
import pdb

import pyautogui

# selenium
import selenium
import pandas as pd
from selenium import webdriver


# BeautifulSoup
from bs4 import BeautifulSoup
import requests

# add time
import time

#to send keys from keyboard
from selenium.webdriver.common.keys import Keys

from selenium.common.exceptions import NoSuchElementException


In [2]:
def open_url(url):
    """function to open the entered url in browser"""
    global driver
    # first, connect to the webdriver
    driver=webdriver.Chrome(r'C:/chromedriver.exe')
    # maximize window
    driver.maximize_window()
    #enter the url
    driver.get(url)
    time.sleep(3)
    # handle 'Privacy Error' from Chrome
    try:
        driver.find_element_by_xpath("//div[@class='interstitial-wrapper']//following::button[@id='details-button']").click()
        
        time.sleep(2)
        
        driver.find_element_by_xpath("//*[@id='proceed-link']").click()
        
        time.sleep(2)
        
    except NoSuchElementException:
        pass

### Q.1. amazon.in , any product entered by user should get searched.

In [3]:
product_name=input("Which amazon product do you have in mind ?:")

url='https://www.amazon.in'

open_url(url)

# give some time to load the webpage
time.sleep(2)

# get web element for amazon search bar
search_wbe=driver.find_element_by_xpath("//input[@class='nav-input nav-progressive-attribute']")
#enter the product into search bar
search_wbe.send_keys(product_name,Keys.ENTER)

# give some time to loda the search results
time.sleep(2)

Which amazon product do you have in mind ?:guiter


### Q.2 - In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a dataframe and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product vertical. 
## Details to be scraped are:    "Brand Name", "Name of the Product", "Rating", "No. of Ratings", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Other Details" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.


In [4]:
def amzn_search(input_prompt):
    """this function will search a particular product category on amazon.in website"""
    
    open_url('https://www.amazon.in')

    # give some time to load the webpage
    time.sleep(3)

    # get web element for amazon search bar
    search_wbe=driver.find_element_by_xpath("//input[@class='nav-input nav-progressive-attribute']")
    #enter the product into search bar
    search_wbe.send_keys(str(input_prompt),Keys.ENTER) 

    # give some time to loda the search results
    time.sleep(3)

In [5]:
def fetch_product():
    # first get all product tiles from current webpage
    #get product tiles webelements
    global tiles_wbe
    tiles_wbe=[]
    tiles_wbe.clear()
    
    tiles_wbe=driver.find_elements_by_xpath("//div[contains(@data-asin,'B0') and @data-component-type='s-search-result']")
    
    # click on each tile one by one and get product info
    for i in range(0,len(tiles_wbe)):
        
        temp_wbe=tiles_wbe[i]
        #add time
        time.sleep(2)
        temp_wbe.click()
        try:
            # switch to new window
            driver.switch_to.window(driver.window_handles[1])
        except IndexError:
            temp_wbe=tiles_wbe[i]
            time.sleep(2)
            temp_wbe.click()
            driver.switch_to.window(driver.window_handles[1])
        # add timer
        time.sleep(3)
        
        # below part is to enter pincode 
        #=========================================================
        # get "enter pincode" button webelement
        pincode_wbe=driver.find_element_by_xpath("//div[@id='contextualIngressPtLabel_deliveryShortLine']")
        
        if pincode_wbe.text=='Select delivery location':
            #click on 'enter pincode' button
            pincode_wbe.click()
            #add time to load pincode sub-window
            time.sleep(2)
            #get the webelement for pincode entry bar
            pin_bar_wbe=driver.find_element_by_xpath("//input[@class='GLUX_Full_Width a-declarative']")
            #enter into pincode bar
            pin_bar_wbe.send_keys('110091')
            #get webelement for pincode 'Apply' button
            apply_btn=driver.find_element_by_xpath("//input[@aria-labelledby='GLUXZipUpdate-announce']")
            #click on pincode 'Apply' button
            apply_btn.click()
            #add time to load the webpage after pincode enter
            time.sleep(3)
          #=====================================================  
        
        time.sleep(3)
        # calling the function to fetch data from current webpage
        fetch_data()
        
        # give some time to fetch data
        time.sleep(3)
        
        # close the particular product tab
        driver.close()
        
        # switch driver to main window with search results
        driver.switch_to.window(driver.window_handles[0])
        
        time.sleep(2)

In [6]:
def fetch_data():
    """this function gets required data from the current product webpage"""
    
    # get brand name
    brand_wbe=driver.find_element_by_xpath("//span[@id='productTitle']")
    brands.append(brand_wbe.text.split(' ')[0])
    # get name of the product
    products.append(brand_wbe.text)
    # get the rating
    try:
        rating_wbe=driver.find_element_by_xpath("//span[@data-hook='rating-out-of-text']") # only first instance of webelement from this page is required
        ratings.append(rating_wbe.text)
    except NoSuchElementException:
        ratings.append('--')

    # get number of ratings
    try:
        number_ratings_wbe=driver.find_element_by_xpath("//a[@id='acrCustomerReviewLink']//following::span")
        number_ratings.append(number_ratings_wbe.text.split(" ")[0])
    except NoSuchElementException:
        number_ratings.append('--')
 
    # get price 
    try:
        price_wbe=driver.find_element_by_xpath("//span[@id='priceblock_ourprice']")
        prices.append(price_wbe.text)
    except NoSuchElementException:
        try:
            price_wbe=driver.find_element_by_xpath("//span[@id='priceblock_dealprice']")
            prices.append(price_wbe.text+"(Deal price)")
        except NoSuchElementException:
            try:
                price_wbe=driver.find_element_by_xpath("//span[@class='a-price-whole']")
                prices.append(price_wbe.text+"(Prime member price)")
            except NoSuchElementException:
                prices.append('--')
    

    # get return/exchange data
    try:
        xchange_wbe=driver.find_element_by_xpath("//a[@class='a-size-small a-link-normal a-text-normal']")
        xchanges.append(xchange_wbe.text)
    except NoSuchElementException:
        xchanges.append('--')
         

    # get expected delivery webelement
    try:
        xp_del_wbe=driver.find_element_by_xpath("//div[@id='ddmDeliveryMessage']")
        if xp_del_wbe.text!='':
            xp_deliveries.append(xp_del_wbe.text)
        else:
            xp_deliveries.append("Currently not available")
    except NoSuchElementException:
        xp_deliveries.append('--')
         

    # get product availability
    try:
        prod_avail_wbe=driver.find_element_by_xpath("//div[@id='availability']")
        prod_avail.append(prod_avail_wbe.text)
    except NoSuchElementException:
        prod_avail.append('--')     

    # get other details
    try:
        other_details_wbe=driver.find_element_by_xpath("//div[@id='featurebullets_feature_div']")
        other_details.append(other_details_wbe.text.replace("\n","."))
    except NoSuchElementException:
        other_details.append('--')
   
    # get product url
    prod_url.append(driver.current_url)


In [7]:
brands=[]
brands.clear()

products=[]
products.clear()

ratings=[]
ratings.clear()

number_ratings=[]
number_ratings.clear()

prices=[]
prices.clear()

xchanges=[]
xchanges.clear()

xp_deliveries=[]
xp_deliveries.clear()

prod_avail=[]
prod_avail.clear()

other_details=[]
other_details.clear()

prod_url=[]
prod_url.clear()

page_urls=[]
page_urls.clear()
input_prompt=input("Which amazon product do you have in mind ?:")

amzn_search(input_prompt)

#add time
time.sleep(3)

# get web element for webpage number button
page_wbe=driver.find_elements_by_xpath("//ul[@class='a-pagination']//following::a")

# get urls of 1st 3 pages
for i in page_wbe:
    temp_url= i.get_attribute('href')
    page_urls.append(temp_url)
    if len(page_urls)==3:
        break

counter=1
# get product info by iterating over each page
for url in page_urls:
    if counter == 1: # first page check
        time.sleep(2)
        fetch_product() # get webelemets for product titles, then click on each title and open the product in new tab and get info.
        counter=2
    elif counter != 1: # first page scraping complete
        time.sleep(2)
        # open the next webpage in current tab
        driver.get(url)
        
        # give time to load
        time.sleep(3)
        
        # fetch data from current webpage
        fetch_product()
        
        # time
        time.sleep(3)

Which amazon product do you have in mind ?:guiter


In [8]:
len(brands)

152

In [9]:
amzn_3pg=pd.DataFrame({})
amzn_3pg['Brand']=brands
amzn_3pg['Product']=products
amzn_3pg['Rating']=ratings
amzn_3pg['Number of ratings']=number_ratings
amzn_3pg['Price']=prices
amzn_3pg['Replacement']=xchanges
amzn_3pg['Expected delivery']=xp_deliveries
amzn_3pg['Product availability']=prod_avail
amzn_3pg['Product description']=other_details
amzn_3pg['Product link']=prod_url

In [10]:
pd.set_option('display.max_colwidth',None)

In [11]:
pd.set_option('display.max_rows',None)

In [12]:
amzn_3pg

,Brand,Product,Rating,Number of ratings,Price,Replacement,Expected delivery,Product availability,Product description,Product link
0,Juârez,"Juârez Acoustic Guitar, 38 Inch Cutaway, 038C with Bag, Strings, Pick and Strap, Black",3.9 out of 5,"10,767","₹ 2,499.00",Pay on Delivery,"FREE delivery: Sunday, July 11 Details",In stock.,"About this item.Black Glossy Finish, Number of Frets: 18, Acoustic Guitar with strap, Bag, Strings and 2 Picks.Great looks with an innovative design to produce good quality sound.Finger board: Linden Wood, Fretboard - Ebony Wood, Size: 38 inches, Cutaway.Included components: Acoustic Guitar with strap, Bag, Strings and 2 Picks.Linden Binding and Full Wood Construction with Geared Tuning, Wood Frame and Steel Strings",https://www.amazon.in/Juarez-Acoustic-Cutaway-038C-Strings/dp/B017NPCSLI/ref=sr_1_1?dchild=1&keywords=guiter&qid=1625502269&sr=8-1
1,Juarez,"Juarez JRZ38C Acoustic Guitar, 38 Inch Cutaway with Bag (TBS Transparent Blue Sunburst)",4 out of 5,"3,679","₹ 2,499.00",Pay on Delivery,"FREE delivery: Sunday, July 11 Details",In stock.,"About this item.Black glossy finish, number of frets: 18, acoustic guitar with strap, bag, strings and 2 picks.Great looks with an innovative design to produce good quality sound.Finger board: Linden wood, fretboard - Ebony wood, size: 38 inches, cutaway.Included components: Acoustic Guitar with strap, Bag, Strings and 2 Picks.Linden binding and full wood construction with geared tuning, wood frame and steel strings.An able acoustic guitar, its neck material is made from Linden Wood, bridge material is made from plastic, finger board and back material is made from Linden Wood, string is made from nylon, and sides material is made from Linden",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-Sunburst-Strings/dp/B01FYYTDS6/ref=sr_1_2?dchild=1&keywords=guiter&qid=1625502269&sr=8-2
2,Giuson,"Giuson Venus Black 41 Inch Rosewood Fretboard Acoustic Guitar With Bag,Strap,1 Set of Extra Strings and 2 Picks by K-Retail",4 out of 5,205,"₹ 3,790.00",Pay on Delivery,"FREE delivery: Saturday, July 10 Details",In stock.,About this item.Product Sold by K-Retail.› See more product details,https://www.amazon.in/Giuson-Venus-Black-Acoustic-Strings/dp/B089GNM189/ref=sr_1_3?dchild=1&keywords=guiter&qid=1625502269&sr=8-3
3,Juarez,"Juarez Acoustic Guitar, 38 Inch Cutaway with Pick Guard, 38CPG with Bag, Strings, Pick and Strap, Black",3.7 out of 5,226,"₹ 2,395.00",Pay on Delivery,"FREE delivery: Sunday, July 11 Details",In stock.,"About this item.Black Glossy Finish, Number of Frets: 18, Acoustic Guitar with strap, Bag, Strings and 2 Picks.Great looks with an innovative design to produce good quality sound.Finger board: Linden Wood, Fretboard - Ebony Wood, Size: 38 inches, Cutaway.Included components: Acoustic Guitar with strap, Bag, Strings and 2 Picks.Linden Binding and Full Wood Construction with Geared Tuning, Wood Frame and Steel Strings",https://www.amazon.in/Juarez-Acoustic-Guitar-Cutaway-Strings/dp/B075XMRKMY/ref=sr_1_4?dchild=1&keywords=guiter&qid=1625502269&sr=8-4
4,JUAREZ,"JUAREZ JRZ38C Right Handed Acoustic Guitar with Bag, Strings, Pick and Strap (Natural, 6 Strings)",4 out of 5,"2,432","₹ 2,499.00",Pay on Delivery,"FREE delivery: Sunday, July 11 Details",In stock.,"About this item.Number of Frets: 18.Acoustic guitar with strap, bag, strings and 2 picks.Great looks with an innovative design to produce quality sound.Finger Board: Linden wood.Fretboard: Ebony wood.Size: 38 inches, cutaway.Linden binding and full wood construction with geared tuning, wood frame and steel strings",https://www.amazon.in/JUAREZ-JRZ38C-Acoustic-Natural-Strings/dp/B01FYYTDWC/ref=sr_1_5?dchild=1&keywords=guiter&qid=1625502269&sr=8-5
5,JUAREZ,"JUAREZ JRZ38C Right Handed Acoustic Guitar (Sunburst, 6 Strings)",4 out of 5,"3,732","₹ 2,499.00",Pay on Delivery,FREE delivery: July 14 - 16 Details,"In stock on July 12, 2021.\nOrder it now.","About this item.Number of Frets: 18.Acoustic guitar with strap, bag, str

In [13]:
amzn_3pg.to_csv("guitars_from_Amazon_India.csv",float_format='%.2f',index=False)

## 3. Write a python program to access the search bar and search button on images.google.com and scrape 100 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’

In [14]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('fruits',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/fruit_pics/fruits_'+str(i)+'.png')

## car pictures

In [15]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('cars',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/car_pics/car_'+str(i)+'.png')

## Machine Learning pics

In [16]:
open_url('https://www.google.co.in/imghp?hl=en&ogbl')

time.sleep(5)
# get web element for search button
search_wbe=driver.find_element_by_xpath("//div[@class='pR49Ae gsfi']//following::input")
#enter text into search bar
search_wbe.send_keys('Machine Learning',Keys.ENTER)
# give time to load the images page
time.sleep(5)
# get webelement for 1st image
first_image_wbe=driver.find_element_by_xpath("//div[@class='bRMDJf islir']//following::img")
# get web element for 'more results' buttom
more_btn=driver.find_element_by_xpath("//div[@class='qvfT1']")
# scroll to 'more results' button
for i in range(5):
    driver.execute_script("arguments[0].scrollIntoView(true);",more_btn)
    time.sleep(10)
# get to the top of page
driver.execute_script("arguments[0].scrollIntoView(true);",first_image_wbe)
# initiate list before using it
images=[]
images.clear()
# get webelemnts for images on the page
images = driver.find_elements_by_xpath("//div[@class='bRMDJf islir']//following::img")
# loop for 100 webelements
for i in range(100):
    images[i].screenshot('F:/ml_pics/ml_'+str(i)+'.png')

## 4. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Display Resolution”, “Processor”, “Processor Cores”, “Battery Capacity”, “Price”, “Product URL”. In case if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [17]:
def fetch_data():
    
    # get product title webelement
    title_wbe=driver.find_element_by_xpath("//span[@class='B_NuCI']")
    # append brand name
    brands.append(title_wbe.text.split(" ")[0])
    # append product name
    products.append(title_wbe.text.split(',')[0].replace("(",""))
    
    # get price webelement
    price_wbe=driver.find_element_by_xpath("//div[@class='_30jeq3 _16Jk6d']")
    # append price
    prices.append(price_wbe.text)
    
    # get product url
    urls.append(driver.current_url)
            
    #get "specifications" text webelement which has all features in text
    features_wbe=driver.find_elements_by_xpath("//div[@class='_3k-BhJ']")
    # get text from all types of 'specifications' in one place
    for wbe in features_wbe:
        features.append(wbe.text.replace("\n","**").replace(",","^^"))
    
    # looping through all specification categories
    for j in range(0,len(features)):
        # check if specification category is 'General'
        if (features[j].split("**"))[0].lower()=='general':
            general=features[j].split("**")
            general_dict={}
            for i in general:
                if general.index(i)%2!=0:
                    general_dict[i.lower()]=general[general.index(i)+1]
            color.append(general_dict['color'])
        # check if specification category is 'Display Features'   
        elif (features[j].split("**"))[0].lower()=='display features':
            display=features[j].split("**")
            display_dict={}
            for i in display:
                if display.index(i)%2!=0:
                    display_dict[i.lower()]=display[display.index(i)+1]
            display_size.append(str(display_dict['display size']))
            display_reso.append(str(display_dict['resolution']))
        # check if specification category is 'OS & Processor Features'
        elif (features[j].split("**"))[0].lower()=='os & processor features':
            processor=features[j].split("**")
            processor_dict={}
            for i in processor:
                if processor.index(i)%2!=0:
                    processor_dict[i.lower()]=processor[processor.index(i)+1]
            if 'processor core' in processor_dict.keys(): 
                os_processors.append(str(processor_dict['operating system'])+str(processor_dict['processor core']))
            else:
                os_processors.append(str(processor_dict['operating system']))
        # check if specification category is 'Memory & Storage Features'
        elif (features[j].split("**"))[0].lower()=='memory & storage features':
            memory=features[j].split("**")
            memory_dict={}
            for i in memory:
                if memory.index(i)%2!=0:
                    memory_dict[i.lower()]=memory[memory.index(i)+1] 
            RAM.append(str(memory_dict['ram']))
            storage_ROM.append(str(memory_dict['internal storage']))
        # check if specification category is 'Camera Features'    
        elif (features[j].split("**"))[0].lower()=='camera features':
            cameras=features[j].split("**")
            camera_dict={}
            for i in cameras:
                if cameras.index(i)%2!=0:
                    camera_dict[i.lower()]=cameras[cameras.index(i)+1]
            primary_cameras.append(str(camera_dict['primary camera']))
            if 'secondary camera' in camera_dict.keys():
                secondary_cameras.append(str(camera_dict['secondary camera']))
            else:
                secondary_cameras.append(str("--"))
        # check if specification category is 'Battery & Power Features'    
        elif (features[j].split("**"))[0].lower()=='battery & power features':
            batt=features[j].split("**")
            batt_dict={}
            for i in batt:
                if batt.index(i)%2!=0:
                    batt_dict[i.lower()]=batt[batt.index(i)+1]
            battery.append(str(batt_dict['battery capacity']))

    features.clear()

In [18]:
open_url("https://www.flipkart.com")

# get web element for login email search bar
email_wbe=driver.find_element_by_xpath("//input[contains(@class,'_2IX_2-')]")#@class='_2IX_2- _2LYh3d VJZDxU'
# enter the email id
email_wbe.send_keys("8208507760")

#get web element for password bar
pwd_wbe=driver.find_element_by_xpath("//input[@type='password']")#class='_2IX_2- _3mctLh VJZDxU'
# enter the password
pwd_wbe.send_keys("Newpassword@72")

# get the webelement for login button
login_btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _2HKlqd _3AWRsL']")
#click on login button
login_btn.click()

#add timer sleep
time.sleep(3)

# get the web element for search bar
search_wbe=driver.find_element_by_xpath("//input[@name='q' and @placeholder='Search for products, brands and more']")
# write into search bar and pressing ENTER 
search_wbe.send_keys("Nokia 7.1 plus",Keys.ENTER)

#add timer sleep
time.sleep(3)


# get webelements for each result tile
result_wbe=driver.find_elements_by_xpath("//div[@class='_2kHMtA']//following::a")

urls=[]
urls.clear()
brands=[]
brands.clear()
products=[]
products.clear()
features=[]
features.clear()
prices=[]
prices.clear()
color=[]
color.clear()
display_size=[]
display_size.clear()
display_reso=[]
display_reso.clear()
os_processors=[]
os_processors.clear()
RAM=[]
RAM.clear()
storage_ROM=[]
storage_ROM.clear()
primary_cameras=[]
primary_cameras.clear()
secondary_cameras=[]
secondary_cameras.clear()
battery=[]
battery.clear()


for product_tile in result_wbe:
    #click on the product_tile one by one
    product_tile.click()
    
    #add time
    time.sleep(2)
    
    #check if new tab has opened
    if len(driver.window_handles)>1:    
        #switch driver to newopened tab
        driver.switch_to.window(driver.window_handles[1])
    else:
        break
    
    #click on the 'Read More' button
    read_more_btn=driver.find_element_by_xpath("//button[@class='_2KpZ6l _1FH0tX']")
    read_more_btn.click()
    
    # add time
    time.sleep(2)
    
    fetch_data()
    
    # close the newly opened tab
    driver.close()
    
    # switch to main search page
    driver.switch_to.window(driver.window_handles[0])
    
    # add time
    time.sleep(2)

In [19]:
temp_df=pd.DataFrame({})
temp_df['Brand']=brands
temp_df['Product']=products
temp_df['Price']=prices
temp_df['Color']=color
temp_df['Display size']=display_size
temp_df['Resolution']=display_reso
temp_df['Operating system & processors']=os_processors
temp_df['RAM']=RAM
temp_df['Storage']=storage_ROM
temp_df['Primary Camera']=primary_cameras
temp_df['Secondary camera']=secondary_cameras
temp_df['Battery']=battery

In [20]:
temp_df

,Brand,Product,Price,Color,Display size,Resolution,Operating system & processors,RAM,Storage,Primary Camera,Secondary camera,Battery
0,Nokia,Nokia 7 Plus Black & Copper,"₹12,000",Black & Copper,15.24 cm (6 inch),2160 x 1080 Pixels,Android Oreo 8Octa Core,4 GB,64 GB,13MP Rear Camera,16MP Front Camera,3800 mAh
1,Nokia,Nokia 6.1 Plus White,"₹18,599",White,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
2,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
3,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
4,Nokia,Nokia 5.1 Plus Black,"₹9,990",Black,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh
5,Nokia,Nokia 6.1 Plus Black,"₹11,490",Black,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,6 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
6,Nokia,Nokia 5.1 Plus Blue,"₹9,975",Blue,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh
7,Nokia,Nokia 5.1 Plus Black,"₹14,999",Black,14.73 cm (5.8 inch),720 x 1520$$pixel,Android Oreo 8.1,6 GB,64 GB,13MP Rear Camera,--,3060 mAh
8,Nokia,Nokia 5.1 Plus White,"₹13,199",White,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
9,Nokia,Nokia 6.1 Plus Blue,"₹18,599",Blue,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh


In [21]:
temp_df.to_csv("Nokia 7.1 plus models.csv",float_format='%.2f',index=False)

In [22]:
pd.read_csv('Nokia 7.1 plus models.csv')

,Brand,Product,Price,Color,Display size,Resolution,Operating system & processors,RAM,Storage,Primary Camera,Secondary camera,Battery
0,Nokia,Nokia 7 Plus Black & Copper,"₹12,000",Black & Copper,15.24 cm (6 inch),2160 x 1080 Pixels,Android Oreo 8Octa Core,4 GB,64 GB,13MP Rear Camera,16MP Front Camera,3800 mAh
1,Nokia,Nokia 6.1 Plus White,"₹18,599",White,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
2,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
3,Nokia,Nokia 5.1 Plus Black,"₹13,199",Black,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
4,Nokia,Nokia 5.1 Plus Black,"₹9,990",Black,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh
5,Nokia,Nokia 6.1 Plus Black,"₹11,490",Black,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,6 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh
6,Nokia,Nokia 5.1 Plus Blue,"₹9,975",Blue,14.73 cm (5.8 inch),720 x 1520$$Pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,13MP + 5MP,8MP Front Camera,3060 mAh
7,Nokia,Nokia 5.1 Plus Black,"₹14,999",Black,14.73 cm (5.8 inch),720 x 1520$$pixel,Android Oreo 8.1,6 GB,64 GB,13MP Rear Camera,--,3060 mAh
8,Nokia,Nokia 5.1 Plus White,"₹13,199",White,14.73 cm (5.8 inch),720 x 1520 pixels,Android Oreo 8.1.0Octa Core,3 GB,32 GB,13MP + 5MP,8MP Front Camera,3060 mAh
9,Nokia,Nokia 6.1 Plus Blue,"₹18,599",Blue,14.73 cm (5.8 inch),2280 x 1080 pixels,Android Oreo 8.1.0Octa Core,4 GB,64 GB,16MP + 5MP,16MP Front Camera,3060 mAh


## 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [23]:
prompt=input("Enter a location: ")

open_url('https://www.google.com/maps/')

# get search bar webelement
search_bar=driver.find_element_by_xpath('//*[@id="searchboxinput"]')
# send city name
search_bar.send_keys(prompt,Keys.ENTER)
time.sleep(10)
current_url=driver.current_url
longitude=current_url.split('@')[1].split(',')[0]
lattitude=current_url.split('@')[1].split(',')[1]
print(prompt," Longitude: ",longitude,'"N')
print(prompt," Lattitude: ",lattitude,'"E')

Enter a location: Jerusalem
Jerusalem  Longitude:  31.7964453 "N
Jerusalem  Lattitude:  35.1053185 "E


## 6. Write a program to scrap details of all the funding deals for second quarter (i.e. July 20 – September 20) from trak.in.

In [ ]:
open_url('https://www.trak.in/')

time.sleep(7)

#click on 'funding deals' button
try:
    driver.find_element_by_xpath('//*[@id="menu-item-51510"]/a/i').click()
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        pass
pdb.set_trace()
months=[]
months.clear()
headers=[]
headers.clear()

try:
    driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
except NoSuchElementException:
    pass

time.sleep(2)   

# block for scraping July-20 

# select maximum table length
try:
    driver.find_element_by_xpath("//div[@id='tablepress-48_length']//following::select[@name='tablepress-48_length']//following::option[@value='100']").click()
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        pass
    
time.sleep(2)

# get month name
try:
    months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-48']").text)
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        try:
            months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-48']").text)
        except:
            pass
time.sleep(2)            
# get table header for our dataframe column names
headers_wbe=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-48 dataTable no-footer']/thead")
for i in headers_wbe:
    headers.append(i.text)
time.sleep(2)
# get table data
row_data_wbe=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-48 dataTable no-footer']/tbody/tr/td")
    
# block for scraping August-20


# select maximum table length
try:
    driver.find_element_by_xpath("//div[@id='tablepress-49_length']//following::select[@name='tablepress-49_length']//following::option[@value='100']").click()
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        pass
    
time.sleep(2)

# get month name
try:
    months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-49']").text)
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        try:
            months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-49']").text)
        except:
            pass
time.sleep(2)            
# get table header for our dataframe column names
headers_wbe_august=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-49 dataTable no-footer']/thead")
for i in headers_wbe_august:
    headers.append(i.text)
time.sleep(2)
# get table data
row_data_wbe_august=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-49 dataTable no-footer']/tbody/tr/td")


# block for sept-20


# select maximum table length
try:
    driver.find_element_by_xpath("//div[@id='tablepress-50_length']//following::select[@name='tablepress-50_length']//following::option[@value='100']").click()
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        pass
    
time.sleep(2)

# get month name
try:
    months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-50']").text)
except NoSuchElementException:
    try:
        driver.find_element_by_xpath("//div[@id='ad_position_box']//following::div[@id='dismiss-button']").click()
    except NoSuchElementException:
        try:
            months.append(driver.find_element_by_xpath("//h2[@class='tablepress-table-name tablepress-table-name-id-50']").text)
        except:
            pass
time.sleep(2)            
# get table header for our dataframe column names
headers_wbe_sept=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-50 dataTable no-footer']/thead")
for i in headers_wbe_sept:
    headers.append(i.text)
time.sleep(2)
# get table data
row_data_wbe_sept=driver.find_elements_by_xpath("//table[@class='tablepress tablepress-id-50 dataTable no-footer']/tbody/tr/td")

#     driver.close()

In [50]:
amount_july

[]

In [51]:
sr_no_july=[]
date_july=[]
startup_july=[]
industry_vertical_july=[]
sub_vertical_july=[]
city_july=[]
investor_name_july=[]
investor_type_july=[]
amount_july=[]

In [52]:
df_july=pd.DataFrame({})
df_july['Sr.No.']=sr_no_july
df_july['Date']=date_july
df_july['Startup']=startup_july
df_july['Industry/Vertical']=industry_vertical_july
df_july['Sub-Vertical']=sub_vertical_july
df_july['City']=city_july
df_july['Investor Name']=investor_name_july
df_july['Investment Type']=investor_type_july
df_july['Amount']=amount_july

In [53]:
df_july.to_csv('July investments.csv',float_format="%.2f",index=False)

In [54]:
df_july

,Sr.No.,Date,Startup,Industry/Vertical,Sub-Vertical,City,Investor Name,Investment Type,Amount


In [55]:
temp_text=[]
temp_text.clear()
for i in row_data_wbe:
    temp_text.append(i.text)
    if len(temp_text)==9:
        sr_no_july.append(temp_text[0])
        date_july.append(temp_text[1])
        startup_july.append(temp_text[2])
        industry_vertical_july.append(temp_text[3])
        sub_vertical_july.append(temp_text[4])
        city_july.append(temp_text[5])
        investor_name_july.append(temp_text[6])
        investor_type_july.append(temp_text[7])
        amount_july.append(temp_text[8])
        temp_text.clear()

In [56]:
sr_no_aug=[]
date_aug=[]
startup_aug=[]
industry_vertical_aug=[]
sub_vertical_aug=[]
city_aug=[]
investor_name_aug=[]
investor_type_aug=[]
amount_aug=[]

In [57]:
df_august=pd.DataFrame({})
df_august['Sr.No.']=sr_no_aug
df_august['Date']=date_aug
df_august['Startup']=startup_aug
df_august['Industry/Vertical']=industry_vertical_aug
df_august['Sub-Vertical']=sub_vertical_aug
df_august['City']=city_aug
df_august['Investor Name']=investor_name_aug
df_august['Investment Type']=investor_type_aug
df_august['Amount']=amount_aug

In [58]:
df_august.to_csv("August investments.csv",float_format='%.2f',index=False)


In [59]:
df_august


,Sr.No.,Date,Startup,Industry/Vertical,Sub-Vertical,City,Investor Name,Investment Type,Amount


In [60]:
temp_text_aug=[]
temp_text_aug.clear()
for i in row_data_wbe_august:
    temp_text_aug.append(i.text)
    if len(temp_text_aug)==9:
        sr_no_aug.append(temp_text_aug[0])
        date_aug.append(temp_text_aug[1])
        startup_aug.append(temp_text_aug[2])
        industry_vertical_aug.append(temp_text_aug[3])
        sub_vertical_aug.append(temp_text_aug[4])
        city_aug.append(temp_text_aug[5])
        investor_name_aug.append(temp_text_aug[6])
        investor_type_aug.append(temp_text_aug[7])
        amount_aug.append(temp_text_aug[8])
        temp_text_aug.clear()

In [61]:
amount_aug

['32,000,000',
 '23,000,000',
 '1,560,000',
 '30,000,000',
 '1,400,000',
 'upto 15,000,000',
 '6,000,000',
 '10,670,000',
 '16,200,000',
 '5,000,000',
 'NA',
 '2,880,000']

In [62]:
sr_no_sept=[]
date_sept=[]
startup_sept=[]
industry_vertical_sept=[]
sub_vertical_sept=[]
city_sept=[]
investor_name_sept=[]
investor_type_sept=[]
amount_sept=[]

In [63]:
df_sept=pd.DataFrame({})
df_sept['Sr.No.']=sr_no_sept
df_sept['Date']=date_sept
df_sept['Startup']=startup_sept
df_sept['Industry/Vertical']=industry_vertical_sept
df_sept['Sub-Vertical']=sub_vertical_sept
df_sept['City']=city_sept
df_sept['Investor Name']=investor_name_sept
df_sept['Investment Type']=investor_type_sept
df_sept['Amount']=amount_sept

In [64]:
df_sept.to_csv("September investments.csv",float_format='%.2f',index=False)

In [65]:
df_sept

,Sr.No.,Date,Startup,Industry/Vertical,Sub-Vertical,City,Investor Name,Investment Type,Amount


In [66]:
temp_text_sept=[]
temp_text_sept.clear()
for i in row_data_wbe_sept:
    temp_text_sept.append(i.text)
    if len(temp_text_sept)==9:
        sr_no_sept.append(temp_text_sept[0])
        date_sept.append(temp_text_sept[1])
        startup_sept.append(temp_text_sept[2])
        industry_vertical_sept.append(temp_text_sept[3])
        sub_vertical_sept.append(temp_text_sept[4])
        city_sept.append(temp_text_sept[5])
        investor_name_sept.append(temp_text_sept[6])
        investor_type_sept.append(temp_text_sept[7])
        amount_sept.append(temp_text_sept[8])
        temp_text_sept.clear()

## 7. Write a program to scrap all the available details of top 10 gaming laptops from digit.in.

In [34]:
open_url('https://www.digit.in')

# click on 'Best gaming laptops' link
driver.find_element_by_xpath('/html/body/div[3]/div/div[2]/div[2]/div[4]/ul/li[9]/a').click()

time.sleep(3)

# get laptop names
names=[]
names.clear()

names_wbe=driver.find_elements_by_xpath("//h3")

for i in names_wbe:
    names.append(i.text)
    
time.sleep(2)

# get specifications box weblements
spec_wbe=driver.find_elements_by_xpath("//div[@class='Spcs-details']")

specs=[]
specs.clear()

for wbe in spec_wbe:
    specs.append(wbe.text)


In [35]:
names

['ALIENWARE AREA 51M R2',
 'ALIENWARE M15 R3',
 'ASUS ROG STRIX SCAR 15',
 'ASUS ROG ZEPHYRUS G14',
 'LENOVO LEGION 5I',
 'ASUS ROG ZEPHYRUS DUO 15',
 'ACER ASPIRE 7 GAMING']

In [36]:
specs

['SPECIFICATION\nOS : Windows 10 Home\nDisplay : 17.3" (1920 x 1080)\nProcessor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 4.1\nDimension : 27.65 x 402.6 x 319.14\nGraphics Processor : Intel® UHD Graphics 630',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (3840 x 2160)\nProcessor : 10th Generation Intel® Core™ i9-10980HK | NA\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : NA\nDimension : NA\nGraphics Processor : NA\nPrice : ₹341990',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 15.6" (1920 x 1080)\nProcessor : AMD Ryzen™ 9 5900HX | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 2.30\nDimension : 35.4 x 25.9 x 2.26\nGraphics Processor : NVIDIA® GeForce RTX™ 3070',
 'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 14" (1920 x 1080)\nProcessor : AMD 3rd Generation Ryzen 9 | 3.3 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 1.65\nDimension : 32.5 x 22.1 x 1.8\nGraphics Processor : NVIDIA GeForce RTX 2060',
 'SPECIFICATION\nOS : W

In [37]:
specs[0]

'SPECIFICATION\nOS : Windows 10 Home\nDisplay : 17.3" (1920 x 1080)\nProcessor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz\nMemory : 1 TB SSD/16 GBGB DDR4\nWeight : 4.1\nDimension : 27.65 x 402.6 x 319.14\nGraphics Processor : Intel® UHD Graphics 630'

In [38]:
temp_specs=[]
for specs in specs:
    temp_specs.append(specs.split('\n'))
temp_specs[0]

['SPECIFICATION',
 'OS : Windows 10 Home',
 'Display : 17.3" (1920 x 1080)',
 'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
 'Memory : 1 TB SSD/16 GBGB DDR4',
 'Weight : 4.1',
 'Dimension : 27.65 x 402.6 x 319.14',
 'Graphics Processor : Intel® UHD Graphics 630']

In [39]:
temp_specs

[['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 17.3" (1920 x 1080)',
  'Processor : 10th Generation Intel® Core™ i7-10700 | 2.90 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 4.1',
  'Dimension : 27.65 x 402.6 x 319.14',
  'Graphics Processor : Intel® UHD Graphics 630'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (3840 x 2160)',
  'Processor : 10th Generation Intel® Core™ i9-10980HK | NA',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : NA',
  'Dimension : NA',
  'Graphics Processor : NA',
  'Price : ₹341990'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 15.6" (1920 x 1080)',
  'Processor : AMD Ryzen™ 9 5900HX | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'Weight : 2.30',
  'Dimension : 35.4 x 25.9 x 2.26',
  'Graphics Processor : NVIDIA® GeForce RTX™ 3070'],
 ['SPECIFICATION',
  'OS : Windows 10 Home',
  'Display : 14" (1920 x 1080)',
  'Processor : AMD 3rd Generation Ryzen 9 | 3.3 GHz',
  'Memory : 1 TB SSD/16 GBGB DDR4',
  'W

In [40]:
temp_specs[0][3][0:3]

'Pro'

In [41]:
os=[]
dis=[]
processor=[]
memory=[]
weight=[]
dimension=[]
graphic=[]
price=[]

In [42]:
for j in temp_specs:
    if len(j)==8:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                price.append('--')
                continue
    elif len(j)==9:
        for i in j:
            if i[0]=='O':
                os.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dis':
                dis.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pro':
                processor.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='M':
                memory.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='W':
                weight.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Dim':
                dimension.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0]=='G':
                graphic.append(i.split(':')[-1])
                time.sleep(2)
                continue
            if i[0:3]=='Pri':
                price.append(i.split(':')[-1])
                time.sleep(2)
                continue

In [43]:
df_digit=pd.DataFrame({})
df_digit['Model Name']=names
df_digit['Operating System']=os
df_digit['Display']=dis
df_digit['Processor']=processor
df_digit['Memory']=memory
df_digit['Weight']=weight
df_digit['Dimension']=dimension
df_digit['Graphics Processor']=graphic
df_digit['Price']=price

In [44]:
df_digit

,Model Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,--
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,--
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,--
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,₹71990
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,--
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,₹56990


In [45]:
df_digit.to_csv('Best gaming laptops.csv',float_format='%.2f',index=False)

In [46]:
df_temp=pd.read_csv('Best gaming laptops.csv')

In [47]:
df_temp

,Model Name,Operating System,Display,Processor,Memory,Weight,Dimension,Graphics Processor,Price
0,ALIENWARE AREA 51M R2,Windows 10 Home,"17.3"" (1920 x 1080)",10th Generation Intel® Core™ i7-10700 | 2.90 GHz,1 TB SSD/16 GBGB DDR4,4.1,27.65 x 402.6 x 319.14,Intel® UHD Graphics 630,--
1,ALIENWARE M15 R3,Windows 10 Home,"15.6"" (3840 x 2160)",10th Generation Intel® Core™ i9-10980HK | NA,1 TB SSD/16 GBGB DDR4,NA,NA,NA,₹341990
2,ASUS ROG STRIX SCAR 15,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 9 5900HX | 3.3 GHz,1 TB SSD/16 GBGB DDR4,2.30,35.4 x 25.9 x 2.26,NVIDIA® GeForce RTX™ 3070,--
3,ASUS ROG ZEPHYRUS G14,Windows 10 Home,"14"" (1920 x 1080)",AMD 3rd Generation Ryzen 9 | 3.3 GHz,1 TB SSD/16 GBGB DDR4,1.65,32.5 x 22.1 x 1.8,NVIDIA GeForce RTX 2060,--
4,LENOVO LEGION 5I,Windows 10 Pro,"15.6"" (1920 x 1080)",10th Generation Intel® Core™ i5-10300H | 2.50 GHz,1 TB SSD/16 GBGB DDR4,2.3,363.06 x 259.61 x 23.57,NVIDIA® GeForce® GTX 1650 4GB,₹71990
5,ASUS ROG ZEPHYRUS DUO 15,Windows 10,"15.6"" (3840 x 1100)",Intel Core i7 10th Gen 10875H | NA,512 GB SSD/4 GBGB DDR4,2.4,268.30 x 360.00 x 20.90,NVIDIA GeForce RTX 2070 Max-Q,--
6,ACER ASPIRE 7 GAMING,Windows 10 Home,"15.6"" (1920 x 1080)",AMD Ryzen™ 5-5500U hexa-core | NA,512 GB SSD/8 GBGB DDR4,2.15,2.29 x 36.3 x 25.4,NVIDIA® GeForce® GTX 1650,₹56990
